In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import SAITS
from pypots.utils.metrics import cal_mae

In [7]:
data_directory = "D:/GitHub/ece209as_project/data"
df = pd.read_csv(data_directory+"/UCI_HAR_Dataset/train/Inertial_Signals/body_acc_x_train.txt", sep="\t")

C:\Users\RUSHI\AppData\Local\Temp\ipykernel_6076\3344015328.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(data_directory+"/UCI_HAR_Dataset/train/Inertial_Signals/body_acc_x_train.txt", sep="  ")


ParserError: Expected 90 fields in line 15, saw 98. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.

In [6]:
df

,1.8085150e-004 1.0138560e-002 9.2755740e-003 5.0658970e-003 1.0810250e-002 4.0451010e-003 4.7573750e-003 6.2136470e-003 3.3067440e-003 7.5719410e-003 5.4071170e-003 6.2212760e-003 6.8950280e-003 4.6102600e-003 7.3313330e-003 5.0776670e-003 5.7625560e-003 6.6927480e-003 5.4432380e-003 8.2409520e-003 6.5064510e-003 6.5315230e-003 7.4224410e-003 5.7717690e-003 6.2401870e-003 4.6229820e-003 4.8329620e-003 5.4987460e-003 4.3411860e-003 5.0980180e-003 4.2687420e-003 3.0451870e-003 3.2041920e-003 4.5204000e-003 5.1272110e-003 3.5497910e-003 4.2342330e-003 4.4686700e-003 3.5727020e-003 5.1362780e-003 5.3797860e-003 4.4085200e-003 2.2440390e-003 -1.1835980e-003 -6.7953420e-004 2.2813890e-003 3.6537280e-003 2.3126390e-003 -4.0421940e-005 5.3422540e-004 2.9632630e-003 4.3994680e-003 2.7023390e-003 3.4523100e-004 2.9169110e-003 4.9785560e-003 3.4468700e-003 4.3781510e-003 5.0061700e-003 1.9967950e-003 -4.7185960e-004 -7.5556790e-004 -1.5281700e-003 -2.3911530e-003 1.0937520e-003 4.5500770e-003 2.8791730e-003 2.2474280e-003 3.3046830e-003 2.4158860e-003 1.6185590e-003 9.8135230e-004 8.8853300e-006 -3.6276470e-004 -4.8702270e-004 -3.5595820e-004 -2.2916060e-004 -1.3114980e-004 -4.4114800e-004 -1.5651630e-003 -9.2940280e-004 8.7199520e-006 -1.2362210e-003 -9.1124670e-004 -4.3503450e-004 -1.1767940e-003 1.6461360e-004 7.5082800e-004 3.8866280e-004 1.3275170e-003 2.0525250e-003 2.7375310e-003 2.7428930e-003 1.3300370e-003 -3.6699150e-004 -7.2165690e-004 -1.9036020e-003 -4.2942530e-003 -2.5081770e-005 5.2508420e-003 2.4898330e-003 8.1063280e-004 1.1660270e-003 -6.9303070e-004 -7.7242430e-005 5.0368690e-004 -4.9460300e-004 1.5624580e-004 6.4592740e-005 -3.2005660e-004 1.8535200e-004 -3.7490410e-004 -1.0128180e-003 -1.6113700e-003 -1.7649400e-003 -6.3572690e-004 1.0474560e-003 3.0186940e-003 1.4121650e-003 -1.5091400e-003 6.0246010e-005 4.3542250e-004 -8.1853380e-004 2.2765550e-004 -2.9989380e-004 -1.1474840e-003 -2.2226550e-004 1.5755000e-003
0,1.0937520e-003 4.5500770e-003 2.8791730e-0...
1,3.5312660e-003 2.2850630e-003 -4.1975380e-0...
2,-1.7723520e-003 -1.3114480e-003 3.8767950e-0...
3,8.7476850e-005 -2.7191750e-004 1.0221030e-0...
4,5.2516130e-004 -1.8632820e-003 -1.5102310e-0...
...,...
7346,3.8887260e-001 6.1876270e-001 7.7406660e-0...
7347,7.1186430e-001 7.0923710e-001 3.2136770e-0...
7348,-2.2671750e-001 -1.7781690e-001 -1.5086910e-0...
7349,-6.4805970e-002 -7.9933000e-002 -6.8349660e-0...
